# Analiza Wielowymiarowa - zajecia 9 - Analiza skupien   

In [ ]:
from multidim.utils import resolve_stata, load_stata

STATA_PATH, STATA_TYPE = resolve_stata(version = 17, stype = "se")
# make sure they are proper ones
STATA_PATH, STATA_TYPE

In [ ]:
load_stata(STATA_PATH, STATA_TYPE)

In [ ]:
from pystata import stata

In [ ]:
import pandas as pd
import scipy
import sklearn
from sklearn.cluster import KMeans
import numpy as np
from collections import Counter
from collections import defaultdict
from sklearn.metrics import calinski_harabasz_score

### Przyklad 1

Dane i przyklad zostaly pozyczone z podrecznika  
Sophia Rabe-Hesketh i Brian Everitt  
"A Handbook of Statistical Analyses using Stata".  
Dane dotycza miast w Stanach Zjednoczonych Ameryki Polnocnej.

In [ ]:
%stata set seed 1234

In [ ]:
from multidim.datasets import load_uscities

In [ ]:
uscities = load_uscities()
uscities_copy = uscities.copy()

In [ ]:
%%stata -d uscities_copy
// precipitaion = opady
des
sum
// jak widac kazda zmienna ilosciowa jest mierzona na innej skali.
// W celu wyeliminowania wplywu wariancji standaryzujemy zmienne.

Skalowanie zminnej. Normalizacja, Min-Max

In [ ]:
x_names = ["so2", "temp", "manuf", "pop", "wind", "precip", "days"]
def z_score(vec):
    return (vec - vec.mean()) / vec.std()
x_names_s = [c + "_s" for c in x_names]
uscities[x_names_s] = uscities[x_names].apply(z_score, axis = 0)

In [ ]:
%%stata
foreach zmienna of varlist so2 temp manuf pop wind precip days {
    egen `zmienna'_s = std(`zmienna')
}

In [ ]:
%stata su

In [ ]:
uscities.describe().T

In [ ]:
%%stata
// Nie majac przeslanek do ustalenia liczby skupien przeanalizujemy modele
// dla 2 do 6 skupien.

// Uruchamiamy generator liczb losowych dla znalezienia poczatkowego rozwiazania.
set seed 20150413

cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl2_mean)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(3) name(cl3_mean)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(4) name(cl4_mean)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(5) name(cl5_mean)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(6) name(cl6_mean)

In [ ]:
X = uscities[x_names_s]
res = defaultdict(dict)
for i in range(2,6):
    cc = str(i)
    kmeans = KMeans(n_clusters=i, random_state=1234, init = "random").fit(X)
    # print(kmeans.labels_)
    res[cc]["labels"] = kmeans.labels_
    res[cc]["count_labels"] = Counter(sorted(kmeans.labels_))
    dfs = pd.DataFrame(kmeans.cluster_centers_.T)
    dfs.index = x_names_s
    res[cc]["centres"] = dfs
    res[cc]["calinski"] = calinski_harabasz_score(X, kmeans.labels_)

In [ ]:
res

In [ ]:
#%stata help cluster

In [ ]:
%%stata -ret cmean_l
cluster list

Polecenie "cluster stop" oblicza i wyswietla wartosc kryterium Calinskiego-Harabasza  
dla podzialu zbiory danych o zadeklarowanej nazwie

In [ ]:
for c in cmean_l['r(names)'].split(" "):
    stata.run('cluster stop {}'.format(c))

In [ ]:
#%%stata
#forvalues i=2(1)6 {
#  cluster stop cl`i'_mean
#}

In [ ]:
%%stata
// Dla kazdego podzialu mozemy obliczyc statystyki opisowe skupien.
// W ten sposob stwierdzamy, czy rozwiazanie jest sensowne.
bysort cl2_mean: su so2_s temp_s manuf_s pop_s wind_s precip_s days_s

Obecnosc skupien o malej liczbie obserwacji wskazuje na wystepowanie obserwacji nietypowych.  
Warto sie zastanowic, czy takich obserwacji nie pominac.  
Srednie sa malo odporne na obserwacje nietypowe.  
Mozliwe, ze w przypadku tego zbioru analiza oparta o mediany da lepsze rezultaty.

In [ ]:
%stata cluster drop *

In [ ]:
%%stata -eret c_eret_temp -sret c_sret_temp -ret c_ret_temp 
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl2_median)

In [ ]:
c_eret_temp, c_sret_temp, c_ret_temp

In [ ]:
%%stata
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl3_median)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl4_median)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl5_median)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl6_median)

In [ ]:
%%stata -ret cmedian_l
cluster list 

In [ ]:
for c in cmedian_l['r(names)'].split(" "):
    stata.run('cluster stop {}'.format(c))

Warto jest przeprowadzic analize wykorzystujac rozne metryki.  
Celem jest sprawdzenie, czy wyniki nie sa przypadkowe.  
Jezeli rzeczywiscie dane sa pogrupowane to grupowania ukaza sie niezaleznie od wyboru metryki.  
Zmieniamy metryke na miejska.

In [ ]:
%stata cluster drop *

In [ ]:
%%stata
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl2_mean) measure(L1)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(3) name(cl3_mean) measure(L1)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(4) name(cl4_mean) measure(L1)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(5) name(cl5_mean) measure(L1)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(6) name(cl6_mean) measure(L1)

forvalues i=2(1)6 {
cluster stop cl`i'_mean
}

cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl2_median) measure(L1)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(3) name(cl3_median) measure(L1)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(4) name(cl4_median) measure(L1)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(5) name(cl5_median) measure(L1)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(6) name(cl6_median) measure(L1)

forvalues i=2(1)6 {
cluster stop cl`i'_median
}

Zmieniamy metryke na maksimum.

In [ ]:
%stata cluster drop *

In [ ]:
%%stata
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl2_mean) measure(Linfinity)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(3) name(cl3_mean) measure(Linfinity)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(4) name(cl4_mean) measure(Linfinity)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(5) name(cl5_mean) measure(Linfinity)
cluster kmeans so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(6) name(cl6_mean) measure(Linfinity)

forvalues i=2(1)6 {
cluster stop cl`i'_mean
}

cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(2) name(cl2_median) measure(Linfinity)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(3) name(cl3_median) measure(Linfinity)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(4) name(cl4_median) measure(Linfinity)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(5) name(cl5_median) measure(Linfinity)
cluster kmedian so2_s temp_s manuf_s pop_s wind_s precip_s days_s, k(6) name(cl6_median) measure(Linfinity)

forvalues i=2(1)6 {
  cluster stop cl`i'_median
}

Diagnostyka
Silhouete
instalacja pakietu Silhouette

In [ ]:
%stata net search silhouette

In [ ]:
%stata ssc install silhouette

pakiet Silhouete zostal napisany dla Stata 10, nalezy ustawic wczesniejsza wersje niz 17

In [ ]:
%stata version 15

utworzenie macierzy niepodobienstwa

In [ ]:
%stata matrix dissim dist = so2_s temp_s manuf_s pop_s wind_s precip_s days_s, Linfinity

obliczenie miary i wykres jej wartosci

In [ ]:
%stata sort town

In [ ]:
%stata silhouette cl2_median, dist(dist) idvar(town) 

In [ ]:
%stata silhouette cl5_median, dist(dist) idvar(town) 

In [ ]:
%stata version 17

In [ ]:
from yellowbrick.cluster import silhouette_visualizer

silhouette_visualizer(KMeans(5), X, colors='yellowbrick')

### Przykład 2

Jest to przyklad bazujacy na artykule Herczynski i Strawinski (2014)  
"Postawy zawodowe nauczycieli. Próba typologii".

In [ ]:
from multidim.datasets import load_nauczyciele

In [ ]:
nauczyciele = load_nauczyciele()
nauczyciele_copy = nauczyciele.copy()

In [ ]:
%stata clear

In [ ]:
%%stata -d nauczyciele_copy
// Obejrzyjmy dane.
describe
summarize

In [ ]:
nauczyciele.dtypes, nauczyciele.describe().T

Chcemy znalezc grupy wsrod nauczycieli. Wczesniejesze badania pozakaly ze czesc nauczycieli deklaruje,
ze bardzo duzo czasu spedza w szkole; inni godza obowiazki szkole z praca w innym miejscu.
Z drugiej strony nauczyciele o wiekszym stazu pracy czesciej otrzymuja nadgodziny od dyrektora szkoly.
Chcemy na podstawie czasu poswiecanego na 4 czynnnosci zawodowe (przygotowanie lekcji i innych zajec,
prowadzenie innych zajec, sprawdzanie prac) oraz ich stazu pracy pogrupowac nauczycieli.
Prawidlowosc podzialu bedzie weryfikowana na podstawie zmiennych metryczkowych oraz opinii nauczycieli.

In [ ]:
%%stata
// Wybor poczatkowej liczby skupien. Poniewaz mamy dwie cechy nauczycieli, zalozylismy ze beda co najmniej 4.
forvalues i=4(1)9 {
  cluster kmeans staz_c_std czas0205_std, k(`i') name(means_6_`i')
}

In [ ]:
x_names = ["staz_c_std", "czas0205_std"]

In [ ]:
X = nauczyciele[x_names]
res = defaultdict(dict)
for i in range(4,9):
    cc = str(i)
    kmeans = KMeans(n_clusters=i, random_state=1234, init = "random").fit(X)
    # print(kmeans.labels_)
    res[cc]["labels"] = Counter(sorted(kmeans.labels_))
    dfs = pd.DataFrame(kmeans.cluster_centers_.T)
    dfs.index = x_names
    res[cc]["centres"] = dfs
    res[cc]["calinski"] = calinski_harabasz_score(X, kmeans.labels_)

In [ ]:
res

In [ ]:
%%stata
forvalues i=4(1)9 {
  cluster stop means_6_`i'
}

In [ ]:
%%stata
graph twoway (sc czas0205 staz_c, msize(vsmall)), ylabel(, angle(horizontal)) scheme(s1mono)

// Wedlug algorytmu optymalna liczba skupien jest 6, ale dla jednego skupienia nie moglismy znalezc sensownej interpretacji.
// Postanowilismy rozdzielic obserwacje z tego skupienia do innych.
gen tmp=means_6_6
recode tmp (5=.)

set seed 1234
cluster kmeans staz_c_std czas0205_std, k(5) name(means_6_5a) start(group(tmp))


In [ ]:
%%stata
/* rozklady dla definiujacych skupienia. */

bysort means_6_5a: su staz czas0205 czas02_std czas03_std czas04_std czas05_std

/* Opinie nauczycieli. */

bysort means_6_5a: su j4 j6_*

/* Srednie wartosci zmiennych metryczkowowych dla poszczegolnych skupien */

bysort means_6_5a: tab m6
bysort means_6_5a: tab m21

bysort means_6_5a: tab m12
bysort means_6_5a: tab m25a
bysort means_6_5a: tab m30
bysort means_6_5a: tab m27a

In [ ]:
%%stata
bysort means_6_5a: su liczba_dzieci
bysort means_6_5a: tab m1